# Busca em Espaço de Estados: BFS e DFS

**Objetivo do notebook**  
1) Definir o conceito de **Espaço de Estados** e conectar com exemplos reais.  
2) Implementar **BFS** (Busca em Largura) e **DFS** (Busca em Profundidade) com código típico e comentado.  
3) Resolver um **problema prático** usando BFS e DFS, comparando comportamento e resultados.

> **Pré-requisitos sugeridos:** noções básicas de Python, listas/dicionários, funções.


## 1. Conceito: Espaço de Estados

Em Inteligência Artificial clássica (e também em Ciência da Computação), muitos problemas podem ser modelados como uma **busca**.

### 1.1 O que é “Espaço de Estados”?

- Um **estado** é uma descrição completa (ou suficiente) de uma situação do problema.
- O **espaço de estados** é o conjunto de **todos os estados possíveis** que o sistema pode assumir.
- Um problema de busca normalmente é definido por:

1. **Estado inicial**: onde começamos.  
2. **Estados objetivo (goal)**: onde queremos chegar.  
3. **Ações / operadores**: transformações possíveis de um estado para outro.  
4. **Função sucessora**: dado um estado, lista os próximos estados alcançáveis.  
5. (Opcional) **Custo**: custo para aplicar uma ação/transição.

### 1.2 Como isso conecta com o mundo real? (exemplos)

- **GPS/Rotas:**  
  - *Estado*: sua posição (rua, coordenadas).  
  - *Ações*: virar à esquerda/direita, seguir em frente, entrar em uma via.  
  - *Objetivo*: chegar ao destino.

- **Rede social (grafo de amigos):**  
  - *Estado*: um usuário.  
  - *Ações*: ir para um amigo conectado.  
  - *Objetivo*: encontrar “alguém a k conexões” (ex.: amigo de amigo).

- **Planejamento de tarefas (workflow):**  
  - *Estado*: quais tarefas já foram concluídas e recursos disponíveis.  
  - *Ações*: executar uma tarefa permitida.  
  - *Objetivo*: concluir o projeto (todas as tarefas).

> Observação importante: frequentemente desenhamos a busca como uma “árvore”, mas essa árvore é **implícita**: ela é gerada pelas ações possíveis, e não precisa existir como uma árvore armazenada.


## 2. BFS e DFS: ideias principais

### 2.1 BFS (Busca em Largura)
- Explora estados por **camadas** (distância em número de passos).
- Usa uma **fila (queue)**.
- Propriedade útil: em grafos não ponderados, BFS encontra o **caminho com menor número de passos**.

### 2.2 DFS (Busca em Profundidade)
- Explora um caminho “até o fim” antes de voltar.
- Usa uma **pilha (stack)** ou **recursão**.
- Útil quando:
  - você quer **qualquer** solução rapidamente,
  - ou quer explorar profundamente com baixa memória (em algumas variantes),
  - mas **não garante** o menor caminho.


## 3. Implementações típicas (BFS e DFS) com reconstrução de caminho

Para que BFS/DFS sejam úteis em problemas reais, normalmente queremos:
- controlar **visitados** (para evitar ciclos),
- manter um **pai (parent)** para reconstruir o caminho.

Abaixo, implementamos BFS e DFS para grafos representados por dicionário:
`graph[node] = [vizinho1, vizinho2, ...]`


In [ ]:
from collections import deque
from typing import Dict, List, Optional, Tuple, Set

def reconstruct_path(parent: Dict, start, goal) -> List:
    """Reconstrói o caminho do start até o goal usando o mapa parent."""
    if goal not in parent and goal != start:
        return []  # não encontrado

    path = [goal]
    cur = goal
    while cur != start:
        cur = parent[cur]
        path.append(cur)
        
    path.reverse()
    return path

def bfs(graph: Dict, start, goal) -> Tuple[bool, List]:
    """BFS clássico: retorna (encontrou?, caminho)."""
    queue = deque([start])
    visited: Set = {start}
    parent: Dict = {}  # parent[v] = u (de onde viemos para chegar em v)

    while queue:
        u = queue.popleft()

        if u == goal:
            return True, reconstruct_path(parent, start, goal)

        for v in graph.get(u, []):
            if v not in visited:
                visited.add(v)
                parent[v] = u
                queue.append(v)

    return False, []

def dfs_iterative(graph: Dict, start, goal) -> Tuple[bool, List]:
    """DFS iterativo: retorna (encontrou?, caminho)."""
    stack = [start]
    visited: Set = {start}
    parent: Dict = {}

    while stack:
        u = stack.pop()  # LIFO

        if u == goal:
            return True, reconstruct_path(parent, start, goal)

        # Para DFS ficar mais previsível, podemos iterar vizinhos em ordem reversa
        # (assim o primeiro vizinho da lista tende a ser explorado primeiro).
        for v in reversed(graph.get(u, [])):
            if v not in visited:
                visited.add(v)
                parent[v] = u
                stack.append(v)

    return False, []

# Exemplo simples de grafo (não é árvore binária; é um grafo geral)
graph_example = {
    "A": ["B", "C"],
    "B": ["D", "E"],
    "C": ["F"],
    "D": [],
    "E": ["F"],
    "F": []
}

found_bfs, path_bfs = bfs(graph_example, "A", "F")
found_dfs, path_dfs = dfs_iterative(graph_example, "A", "F")

found_bfs, path_bfs, found_dfs, path_dfs


### Interpretação do exemplo acima
- BFS tende a achar um caminho com **menos passos**.
- DFS pode achar um caminho diferente (depende da ordem dos vizinhos), e não garante menor caminho.


## 4. Problema prático: achar caminho em um labirinto (grid)

### 4.1 Modelagem como espaço de estados
- **Estado**: posição do agente no grid `(linha, coluna)`.
- **Ações**: mover `cima`, `baixo`, `esquerda`, `direita` (se não houver parede).
- **Objetivo**: sair do `S` (start) e chegar ao `G` (goal).

Vamos usar uma matriz com:
- `#` = parede
- `.` = caminho livre
- `S` = início
- `G` = objetivo


In [ ]:
from typing import Iterable

maze = [
    list("S..#..."),
    list(".##.#.."),
    list("..#...."),
    list("##.##.."),
    list("...#..G"),
]

ROWS, COLS = len(maze), len(maze[0])

def find_char(grid, ch: str) -> Tuple[int, int]:
    for r in range(len(grid)):
        for c in range(len(grid[0])):
            if grid[r][c] == ch:
                return (r, c)
    raise ValueError(f"Caractere {ch} não encontrado.")

start = find_char(maze, "S")
goal  = find_char(maze, "G")

start, goal


### 4.2 Função sucessora (gerar próximos estados)

Aqui está o ponto-chave do **espaço de estados**: em vez de “ter uma árvore pronta”, nós **geramos** os próximos estados conforme as regras do problema.


In [1]:
def neighbors(pos: Tuple[int,int], grid) -> Iterable[Tuple[int,int]]:
    r, c = pos
    for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:  # cima, baixo, esq, dir
        nr, nc = r + dr, c + dc
        if 0 <= nr < len(grid) and 0 <= nc < len(grid[0]):
            if grid[nr][nc] != '#':  # não atravessa parede
                yield (nr, nc)

# teste rápido
list(neighbors(start, maze))


NameError: name 'Tuple' is not defined

### 4.3 BFS e DFS no labirinto (com reconstrução de caminho)

A lógica é a mesma do grafo, mas agora os “nós” são posições `(r,c)` e os vizinhos vêm da função `neighbors`.


In [ ]:
def reconstruct_path_grid(parent: Dict[Tuple[int,int], Tuple[int,int]], start, goal) -> List[Tuple[int,int]]:
    if goal not in parent and goal != start:
        return []
    path = [goal]
    cur = goal
    while cur != start:
        cur = parent[cur]
        path.append(cur)
    path.reverse()
    return path

def bfs_grid(grid, start, goal):
    q = deque([start])
    visited = {start}
    parent = {}

    while q:
        u = q.popleft()
        if u == goal:
            return True, reconstruct_path_grid(parent, start, goal)

        for v in neighbors(u, grid):
            if v not in visited:
                visited.add(v)
                parent[v] = u
                q.append(v)

    return False, []

def dfs_grid(grid, start, goal):
    stack = [start]
    visited = {start}
    parent = {}

    while stack:
        u = stack.pop()
        if u == goal:
            return True, reconstruct_path_grid(parent, start, goal)

        # ordem reversa para deixar o comportamento mais reprodutível
        for v in reversed(list(neighbors(u, grid))):
            if v not in visited:
                visited.add(v)
                parent[v] = u
                stack.append(v)

    return False, []

found_bfs, path_bfs = bfs_grid(maze, start, goal)
found_dfs, path_dfs = dfs_grid(maze, start, goal)

len(path_bfs), len(path_dfs), found_bfs, found_dfs


### 4.4 Visualização do caminho encontrado

Vamos imprimir o grid marcando o caminho:
- `*` para o caminho (exceto S e G)


In [ ]:
def draw_path(grid, path: List[Tuple[int,int]]) -> List[List[str]]:
    g = [row[:] for row in grid]
    for (r,c) in path:
        if g[r][c] not in ("S", "G"):
            g[r][c] = "*"
    return g

def print_grid(grid):
    for row in grid:
        print("".join(row))

print("Labirinto original:")
print_grid(maze)

print("\nCaminho BFS (menor nº de passos):")
print_grid(draw_path(maze, path_bfs))

print("\nCaminho DFS (depende da ordem de exploração):")
print_grid(draw_path(maze, path_dfs))


## 5. O que aprender com o experimento

- **BFS**: tende a encontrar o caminho com menos passos (ótimo em grafos não ponderados).  
- **DFS**: pode encontrar um caminho mais longo, mas às vezes chega rápido a uma solução (depende do problema e da ordem de vizinhos).  
- Em IA, isso é “busca em espaço de estados”: o “grafo” pode ser implícito e gerado pela função sucessora.